In [ ]:
!pip install transformers fastapi nest-asyncio pyngrok uvicorn
import os
import json
import torch
import numpy as np
import hashlib
import uvicorn
import nest_asyncio
import threading
from typing import List, Optional
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList

CONFIG = {
    "keys": [
        654, 123, 876, 345, 987, 234, 765, 432,
        111, 222, 333, 444, 555, 666, 777, 888, 999, 1010, 2020, 3030
    ],
    "ngram_len": 5,
    "watermark_bias": 2.0
}

def inject_invisible_watermark(text: str) -> str:
    """
    Injects a zero-width space (\u200b) after every 3rd word.
    This is invisible to the user but detectable by code.
    """
    words = text.split(" ")
    # We create a new list to avoid modifying while iterating logic issues
    new_words = []
    for i, word in enumerate(words):
        new_words.append(word)
        # Add marker every 3 words (index 2, 5, 8...)
        if (i + 1) % 3 == 0:
            new_words[-1] = new_words[-1] + "\u200b"

    return " ".join(new_words)

def compute_g_values_for_token(ngram_ids: List[int], next_token_id: int, key: int) -> float:
    data = f"{ngram_ids}-{next_token_id}-{key}".encode('utf-8')
    hash_bytes = hashlib.sha256(data).digest()
    hash_int = int.from_bytes(hash_bytes[:4], 'big')
    return hash_int / (2**32 - 1)

def numpy_weighted_mean_score(g_values: np.ndarray, mask: np.ndarray, weights: Optional[np.ndarray] = None) -> float:
    watermarking_depth = g_values.shape[-1]
    if weights is None: weights = np.ones(watermarking_depth)
    weights = weights * (watermarking_depth / np.sum(weights))
    weighted_g = g_values * weights
    score_per_token = np.sum(weighted_g, axis=1) / watermarking_depth
    num_unmasked = np.sum(mask)
    if num_unmasked == 0: return 0.0
    total_score = np.sum(score_per_token * mask)
    return float(total_score / num_unmasked)

class CustomDetector:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.ngram_len = CONFIG["ngram_len"]
        self.keys = CONFIG["keys"]

    def score(self, text: str) -> float:
        # Standard Statistical Scoring
        ids = self.tokenizer(text, add_special_tokens=True)['input_ids']
        if len(ids) <= self.ngram_len: return 0.0
        seq_len = len(ids) - self.ngram_len

        g_vals = []
        mask = []

        for i in range(seq_len):
            current_token_idx = i + self.ngram_len
            context_ngram = ids[i : i+self.ngram_len]
            token = ids[current_token_idx]

            row_g_vals = []
            for key in self.keys:
                row_g_vals.append(compute_g_values_for_token(context_ngram, token, key))
            g_vals.append(row_g_vals)
            mask.append(1.0)

        if not g_vals: return 0.0
        return numpy_weighted_mean_score(np.array(g_vals), np.array(mask))

class PurePythonWatermarkLogitsProcessor(LogitsProcessor):
    def __init__(self, ngram_len, keys, bias):
        self.ngram_len = ngram_len
        self.keys = keys
        self.bias = bias

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        current_ids = input_ids[0].tolist()
        if len(current_ids) < self.ngram_len: return scores
        ngram = current_ids[-self.ngram_len:]

        # Optimization: Only check top 50 tokens to save time
        top_values, top_indices = torch.topk(scores, 50, dim=1)

        for idx in top_indices[0]:
            token_id = idx.item()
            g_sum = sum(compute_g_values_for_token(ngram, token_id, key) for key in self.keys)
            avg_g = g_sum / len(self.keys)
            if avg_g > 0.5:
                scores[0, token_id] += (avg_g * self.bias)
        return scores

print("⏳ Loading Model (Gemma-2B)...")
HF_TOKEN = "" # <--- REPLACE
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)

print("⚙️  Initializing Detector Engine...")
detector_engine = CustomDetector(tokenizer)

app = FastAPI()
nest_asyncio.apply()

class Req(BaseModel):
    text: str

@app.post("/prompt")
async def generate(req: Req):
    inputs = tokenizer(req.text, return_tensors="pt").to("cuda")

    clean_out = model.generate(**inputs, max_new_tokens=80, do_sample=True)
    clean_txt = tokenizer.decode(clean_out[0], skip_special_tokens=True)

    wm_processor = PurePythonWatermarkLogitsProcessor(
        ngram_len=CONFIG["ngram_len"],
        keys=CONFIG["keys"],
        bias=CONFIG["watermark_bias"]
    )
    wm_out = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        logits_processor=LogitsProcessorList([wm_processor])
    )
    raw_wm_txt = tokenizer.decode(wm_out[0], skip_special_tokens=True)

    final_wm_txt = inject_invisible_watermark(raw_wm_txt)

    return {"clean": clean_txt, "watermarked": final_wm_txt}

@app.post("/detect")
async def detect(req: Req):
    text = req.text

    # CHECK 1: Invisible Symbol (Hard Match)
    if "\u200b" in text:
        return {
            "score": 1.0,
            "verdict": "Watermarked (Hidden Tag Found)",
            "method": "Deterministic"
        }

    # CHECK 2: Statistical (Fallback)
    score = detector_engine.score(text)

    return {
        "score": score,
        "verdict": "Watermarked" if score > 0.60 else "Clean",
        "method": "Statistical"
    }

NGROK_TOKEN = "" # <--- REPLACE

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

import os
os.system("fuser -k 8000/tcp")
ngrok.kill()

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 API IS LIVE AT: {public_url}")
    print(f"📄 DOCS: {public_url}/docs")
except Exception as e:
    print(f"Error connecting ngrok: {e}")
